In [6]:
import pandas as pd
import numpy as np
import datetime as dt

### Import

In [7]:
url='https://drive.google.com/file/d/1eZmXLBHXDgMjuJ8_ZdIc09my8OB02XnI/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
df

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN
...,...,...,...,...,...,...,...
461768,539991,21618,4 WILDFLOWER BOTANICAL CANDLES,1,2010-12-23 16:49:00,1.25,NaN
461769,539991,72741,GRAND CHOCOLATECANDLE,4,2010-12-23 16:49:00,1.45,NaN
461770,539992,21470,FLOWER VINE RAFFIA FOOD COVER,1,2010-12-23 17:41:00,3.75,NaN
461771,539992,22258,FELT FARM ANIMAL RABBIT,1,2010-12-23 17:41:00,1.25,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461773 entries, 0 to 461772
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      461773 non-null  object 
 1   product_code  461773 non-null  object 
 2   product_name  459055 non-null  object 
 3   quantity      461773 non-null  int64  
 4   order_date    461773 non-null  object 
 5   price         461773 non-null  float64
 6   customer_id   360853 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.7+ MB


### Data Cleansing

In [9]:
df_clean = df.copy()

df_clean['date'] = pd.to_datetime(df_clean['order_date']).dt.date.astype('datetime64[ns]')
df_clean['customer_id'] = df_clean['customer_id'].astype(str)
df_clean['quantity'] = df_clean['quantity'].abs()

df_clean = df_clean[~df_clean['customer_id'].isna()]
df_clean = df_clean[~df_clean['product_name'].isna()]
df_clean = df_clean[df_clean['order_id'].str[:1]!='C']
df_clean = df_clean[df_clean['price']>0]

df_clean['product_name'] = df_clean['product_name'].str.lower()
df_clean = df_clean[(~df_clean['product_code'].str.lower().str.contains('test')) |
                    (~df_clean['product_name'].str.contains('test '))]
df_clean['amount'] = df_clean['quantity'] * df_clean['price']

most_freq_product_name = df_clean.groupby(['product_code','product_name'], as_index=False).agg(order_cnt=('order_id','nunique')).sort_values(['product_code','order_cnt'], ascending=[True,False])
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])
df_clean = df_clean.merge(most_freq_product_name.rename(columns={'product_name':'most_freq_product_name'}), how='left', on='product_code')
df_clean['product_name'] = df_clean['most_freq_product_name']
df_clean = df_clean.drop(columns='most_freq_product_name')

from scipy import stats
df_clean = df_clean[(np.abs(stats.zscore(df_clean[['quantity','amount']]))<3).all(axis=1)]
df_clean = df_clean.reset_index(drop=True)
df_clean

,order_id,product_code,product_name,quantity,order_date,price,customer_id,date,amount
0,493413,21724,panda and bunnies sticker sheet,1,2010-01-04 09:54:00,0.85,nan,2010-01-04,0.85
1,493413,84578,elephant toy with blue t-shirt,1,2010-01-04 09:54:00,3.75,nan,2010-01-04,3.75
2,493413,21723,alphabet hearts sticker sheet,1,2010-01-04 09:54:00,0.85,nan,2010-01-04,0.85
3,493414,21844,red retrospot mug,36,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,91.80
4,493414,21533,retro spot large milk jug,12,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,51.00
...,...,...,...,...,...,...,...,...,...
446833,539991,21618,4 wildflower botanical candles,1,2010-12-23 16:49:00,1.25,nan,2010-12-23,1.25
446834,539991,72741,grand chocolatecandle,4,2010-12-23 16:49:00,1.45,nan,2010-12-23,5.80
446835,539992,21470,flower vine raffia food cover,1,2010-12-23 17:41:00,3.75,nan,2010-12-23,3.75
446836,539992,22258,felt farm animal rabbit,1,2010-12-23 17:41:00,1.25,nan,2010-12-23,1.25


In [10]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 446838 entries, 0 to 446837
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      446838 non-null  object        
 1   product_code  446838 non-null  object        
 2   product_name  446838 non-null  object        
 3   quantity      446838 non-null  int64         
 4   order_date    446838 non-null  object        
 5   price         446838 non-null  float64       
 6   customer_id   446838 non-null  object        
 7   date          446838 non-null  datetime64[ns]
 8   amount        446838 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 30.7+ MB


### Basket Dataframe

In [11]:
basket = pd.pivot_table(df_clean, index='order_id', columns='product_name', values='product_code', aggfunc='nunique', fill_value=0)
basket
#pd.set_option('display.max_column', None)
#pd.reset_option('display.max_column', None)

product_name,*boombox ipod classic,*usb office mirror ball,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493413,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493432,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539988,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539989,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
basket_encode = basket.astype(bool)
basket_encode

product_name,*boombox ipod classic,*usb office mirror ball,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493413,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539988,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539989,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539990,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
basket_filter = basket_encode[(basket_encode>0).sum(axis=1)>1]
basket_filter

product_name,*boombox ipod classic,*usb office mirror ball,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493413,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539986,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539988,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539990,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Apriori Algorithm

In [14]:
#conda install -c conda-forge mlxtend
from mlxtend.frequent_patterns import apriori

frequent_itemset = apriori(basket_filter, min_support=.01, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
frequent_itemset['product_cnt'] = frequent_itemset['itemsets'].apply(lambda x: len(x))
frequent_itemset

,support,itemsets,product_cnt
0,0.181863,(white hanging heart t-light holder),1
1,0.111480,(regency cakestand 3 tier),1
2,0.106558,(jumbo bag red retrospot),1
3,0.092777,(pack of 72 retro spot cake cases),1
4,0.087055,(strawberry ceramic trinket box),1
...,...,...,...
1880,0.010028,(fairy cakes notebook a6 size),1
1881,0.010028,"(set/5 red spotty lid glass bowls, jumbo stora...",2
1882,0.010028,"(paper chain kit vintage christmas, ribbon ree...",2
1883,0.010028,(flower vine raffia food cover),1


In [15]:
from mlxtend.frequent_patterns import association_rules

product_association = association_rules(frequent_itemset, metric='confidence', min_threshold=.7).sort_values(['support','confidence'], ascending=[False,False]).reset_index(drop=True)
product_association

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(sweetheart ceramic trinket box),(strawberry ceramic trinket box),0.056909,0.087055,0.044850,0.788108,9.052939,0.039896,4.308539,0.943216
1,(red hanging heart t-light holder),(white hanging heart t-light holder),0.058078,0.181863,0.041713,0.718220,3.949240,0.031151,2.903464,0.792833
2,(toilet metal sign),(bathroom metal sign),0.030085,0.048726,0.022948,0.762781,15.654350,0.021482,4.010110,0.965157
3,"(jumbo bag strawberry, jumbo bag pink with whi...",(jumbo bag red retrospot),0.025102,0.106558,0.018642,0.742647,6.969391,0.015967,3.471659,0.878569
4,(green regency teacup and saucer),(roses regency teacup and saucer),0.022764,0.023686,0.017842,0.783784,33.089926,0.017303,4.515450,0.992369
...,...,...,...,...,...,...,...,...,...,...
127,"(poppy's playhouse livingroom, poppy's playhou...",(poppy's playhouse kitchen),0.010459,0.020856,0.010090,0.964706,46.254659,0.009872,27.742402,0.988722
128,"(poppy's playhouse kitchen, poppy's playhouse ...",(poppy's playhouse livingroom),0.010644,0.015996,0.010090,0.947977,59.263139,0.009920,18.914742,0.993703
129,(poppy's playhouse bathroom),"(poppy's playhouse kitchen, poppy's playhouse ...",0.011443,0.014089,0.010090,0.881720,62.582899,0.009929,8.335431,0.995412
130,"(red spotty charlotte bag, 60 teatime fairy ca...",(pack of 72 retro spot cake cases),0.012674,0.092777,0.010090,0.796117,8.580953,0.008914,4.449712,0.894803


In [16]:
product_association.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
count,132.000000,132.000000,132.000000,132.000000,132.000000,132.000000,132.000000,132.000000
mean,0.016612,0.053040,0.012872,0.783662,23.278721,0.011930,5.207449,0.946492
std,0.006155,0.033950,0.004416,0.066235,16.355197,0.003689,3.260962,0.045669
min,0.010459,0.011443,0.010090,0.701389,3.941090,0.007667,2.888272,0.757116
25%,0.013689,0.022733,0.010767,0.725092,9.810362,0.009969,3.434064,0.913644
50%,0.015042,0.048726,0.011782,0.769786,15.898451,0.010993,4.090468,0.953832
75%,0.017719,0.087055,0.013412,0.831966,35.176812,0.012525,5.788887,0.986513
max,0.058078,0.181863,0.044850,0.964706,63.390950,0.039896,27.742402,0.998851


Insight:

It can be concluded that when customers buy antecedents, they often also buy consequent products at the same time with a confidence value always above 70%. So that antecedent products can be sold together with consequent products. However, it should be noted that sales of white hanging heart t-light holders have a large difference compared to sales of red hanging heart t-light holders. Therefore, the white hanging heart t-light holder needs to be sold separately in a larger quantity than the bundle amount of red hanging heart t-light holder and white hanging heart t-light holder.